# IMPORT

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf #tensorflow
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load
%matplotlib inline
from matplotlib import pyplot as plt

print(tf.__version__)

1.0.1


# PATH

In [3]:
#path
#DATA_PATH = './cifar10/cifar-10-batches-py/' #python pickle
DATA_PATH = './cifar10/cifar-10-batches-bin/' #binary
CHECKPOINT = 'simple_res_net_0529.ckpt'
SAVE_PATH = './checkpoint/res_0604/'
BOARD_PATH = './tensorboard/board_res_0604/'
TEST_FILE_PATH='./cifar10/cifar-10-batches-py/test_batch'

# PARAMETERS

In [4]:
#parameters

INPUT_SIDE = 32
INPUT_SIZE = INPUT_SIDE * INPUT_SIDE
N_CHANNEL = 3
BATCH_SIZE = 64
EPOCHS = 1000
LR = 0.1
LR_DECAY_RATE = 0.5

LABEL_BYTES = 1 
NUM_EXAMPLES_PER_EPOCH = 50000 #NUMBER OF TRAIN DATA SET
NUM_BATCHES_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH / BATCH_SIZE
NUM_TESTSET_PER_EPOCH = 10000
NUM_TEST_BATCHES_PER_EPOCH = NUM_TESTSET_PER_EPOCH / BATCH_SIZE

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

#loss & accuracy save
TRAIN_OUT_FILE_NAME = 'RES_NET_CIFAR10_0604.log'
train_loss_out = open('TRAIN_LOSS'+TRAIN_OUT_FILE_NAME, 'w')
train_accr_out = open('TRAIN_ACCURACY'+TRAIN_OUT_FILE_NAME, 'w')

TEST_OUT_FILE_NAME = 'RES_NET_CIFAR10_0604.log'
test_loss_out = open('TEST_LOSS'+TEST_OUT_FILE_NAME, 'w')
test_accr_out = open('TEST_ACCURACY'+TEST_OUT_FILE_NAME, 'w')

print("input image size : {}".format(INPUT_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LR))
print("learning decay rate : {}".format(LR_DECAY_RATE))

input image size : 1024
image channel : 3
batch size : 64
num of class : 10
training epochs : 1000
learning rate : 0.1
learning decay rate : 0.5


# FUNCTIONS

In [5]:
#Batch Normalization - tf.nn.batch_normalization - 쓸지 말지 모르지만 우선 넣어둠 냄겨둠. 지금은 tf.layer로 쓰는데 tf.nn으로 쓰고싶다..
def batch_norm(x, n_out, phase_train, scope='bn'):
    """
    Batch normalization on convolutional maps.
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    
    with tf.variable_scope(scope):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)
        if phase_train:
            mean, var = mean_var_with_update()
        else:
            mean = ema.average(batch_mean)
            var = ema.average(batch_var)

        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

#CIFAR 데이터 읽어오기 
def read_dataset(filename_queue):
    class DataRecord(object):
        pass
    
    result = DataRecord()
    
    label_bytes = LABEL_BYTES
    result.height = INPUT_SIDE
    result.width = INPUT_SIDE
    result.depth = N_CHANNEL
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    result.key, value = reader.read(filename_queue)
    print (value)
    record_bytes = tf.decode_raw(value, tf.uint8)
    
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32) #input, begin, end로 잘라냄 > 0~1
    
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], # 1부터 image_bytes까지 image를 잘라낸 후 
                                              [label_bytes + image_bytes]), #channel x height x width로 변환함.
                             [result.depth, result.height, result.width])
    
    result.uint8image = tf.transpose(depth_major, [1,2,0]) #channel x height x width >>> height x width x chennl로 변경
    
    return result

#label, image 세팅해줌 
def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
        in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      shuffle: boolean indicating whether to use a shuffling queue.
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16 #프로세스 스레드 
    if shuffle: #셔플 하면 
        images, label_batch = tf.train.shuffle_batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples) #이거만 해주면 batch와 동일함. 
    #Minimum number elements in the queue after a dequeue, used to ensure a level of mixing of elements.
    else:
        images, label_batch = tf.train.batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images) #걍 보드용인듯

    return images, tf.reshape(label_batch, [batch_size])


def distorted_inputs(data_dir, batch_size): #좀 드럽게 인풋받아오기 
    """Construct distorted input for CIFAR training using the Reader ops.
    Args:
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in range(1, 6)] 
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_dataset(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = INPUT_SIDE
    width = INPUT_SIDE

    # Image processing for training the network.  Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3]) # 랜덤으로 이미지를 크롭 h x w x c

    # Randomly flip the image horizontally.
    distorted_image = tf.image.random_flip_left_right(distorted_image) # 또 랜덤으로 좌우 플립 random_flip_up_down도 있긴 함
    #이런 느낌..? 으로 맥스텔타가지고 유니폼으로 어쩌구저쩌구
    #delta = random_ops.random_uniform([], -max_delta, max_delta, seed=seed)
    #return adjust_brightness(image, delta)

    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    distorted_image = tf.image.random_brightness(distorted_image,    #랜덤으로 밝기 조절 
                                               max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image,      #콘트라스트 조절 랜덤. 기존 픽셀값의 최소 0.2 최대 1.8
                                             lower=0.2, upper=1.8)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(distorted_image) #표준화시킴. mean으로빼고 var로 나눔

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH * min_fraction_of_examples_in_queue)
    print('Filling queue with %d CIFAR images before starting to train. '
         'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=True)


def inputs(eval_data, data_dir, batch_size):
    """Construct input for CIFAR evaluation using the Reader ops.
    Args:
    eval_data: bool, indicating if one should use the train or eval data set.
    data_dir: Path to the CIFAR-10 data directory.
    batch_size: Number of images per batch.
    Returns:
    images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
    labels: Labels. 1D tensor of [batch_size] size.
    """
    if not eval_data:
        filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                     for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH
    else:
        filenames = [os.path.join(data_dir, 'test_batch.bin')]
        num_examples_per_epoch = NUM_TESTSET_PER_EPOCH

    for f in filenames:
        if not tf.gfile.Exists(f):
             raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_dataset(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = INPUT_SIDE
    width = INPUT_SIDE

    # Image processing for evaluation.
    # Crop the central [height, width] of the image.
    resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,
                                                         height, width)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(resized_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH * min_fraction_of_examples_in_queue)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=False)


print("FUNCTIONS READY")

FUNCTIONS READY


# NETWORK PARAMETERS

In [6]:
#NETWORK PARAMETERS

stddev = 0.1

weights = {
    'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 16], stddev=stddev), name='conv'),
    'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev, name='conv1_1x1')),
    'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev, name='conv1_3x3')),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev, name='conv1_1x1_16')),
    'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev, name='conv2_1x1')),
    'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev, name='conv2_3x3')),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev, name='conv2_1x1_16')),
    
    # conv 16 + conv2 16 = 32filters 
    
    'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev, name='conv3_1x1')),
    'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev, name='conv3_3x3')),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev, name='conv3_1x1_32')),
    'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev, name='conv4_1x1')),
    'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev, name='conv4_3x3')),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev, name='conv4_1x1_32')),
    
    # conv2 지난거32 + conv4 32 = 64
    
    'conv5_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev, name='conv5_1x1')),
    'conv5_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev, name='conv5_3x3')),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev, name='conv5_1x1_64')),
    'conv6_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev, name='conv6_1x1')),
    'conv6_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev, name='conv6_3x3')),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev, name='conv6_1x1_64')),
    
    #conv4 지난거 64 + conv6 64 = 128
    
    'dense1' : tf.Variable(tf.random_normal([16*16*128, 1000], stddev=stddev, name='dense1')),
    'dense2' : tf.Variable(tf.random_normal([1000, N_CLASSES], stddev=stddev, name='dense2'))    
}
#conv net biases 현재 사용하지 않음. 
biases = {
    'conv' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv_b')),
    'conv1_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_1x1_b')),
    'conv1_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_3x3_b')),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv1_1x1_16_b')),
    'conv2_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_1x1_b')),
    'conv2_3x3' :tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_3x3_b')),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv2_1x1_16_b')),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_1x1_b')),
    'conv3_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_3x3_b')),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv3_1x1_32_b')),
    'conv4_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_1x1_b')),
    'conv4_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_3x3_b')),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv4_1x1_32_b')),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_1x1_b')),
    'conv5_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_3x3_b')),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv5_1x1_64_b')),
    'conv6_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_1x1_b')),
    'conv6_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_3x3_b')),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv6_1x1_64_b')),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev, name='dense1_b')),
    'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev, name='dense2_b'))
}

# NETWORK MODEL (RESNET)

In [7]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='ResNet', training=False):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        with tf.name_scope('conv') as scope:
            conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
            #conv = tf.nn.bias_add(conv, _b['conv'])
            #conv = batch_norm(conv, conv[0].shape[2], training)#배치노말라이제이션만 해주자 
            conv = tf.layers.batch_normalization(conv, training=training, name='bn_conv')
            conv = tf.nn.relu(conv)
            network['conv'] = conv
        
        #16
        with tf.name_scope('conv2') as scope:
            conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
            conv1_1x1 = tf.nn.relu(conv1_1x1)
            conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
            conv1_3x3 = tf.nn.relu(conv1_3x3)
            conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
            #conv1_1x1_16 = batch_norm(conv1_1x1_16, conv1_1x1_16[0].shape[2], training)
            conv1_1x1_16 = tf.layers.batch_normalization(conv1_1x1_16, training=training, name='bn_conv1')
            conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
            network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        with tf.name_scope('conv3') as scope:
            conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
            conv2_1x1 = tf.nn.relu(conv2_1x1)
            conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
            conv2_3x3 = tf.nn.relu(conv2_3x3)
            conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
            #conv2_1x1_16 = batch_norm(conv2_1x1_16, conv2_1x1_16[0].shape[2], training)
            conv2_1x1_16 = tf.layers.batch_normalization(conv2_1x1_16, training=training, name='bn_conv2')
            #print(conv2_1x1_16.shape)
            #32
            conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
            conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
            network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        with tf.name_scope('conv4') as scope:
            conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
            conv3_1x1 = tf.nn.relu(conv3_1x1)
            conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
            conv3_3x3 = tf.nn.relu(conv3_3x3)
            conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
            #conv3_1x1_32 = batch_norm(conv3_1x1_32, conv3_1x1_32[0].shape[2], training)
            conv3_1x1_32 = tf.layers.batch_normalization(conv3_1x1_32, training=training, name='bn_conv3')
            conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
            network['conv3_1x1_32'] = conv3_1x1_32
        
        with tf.name_scope('conv5') as scope:
            conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
            conv4_1x1 = tf.nn.relu(conv4_1x1)
            conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
            conv4_3x3 = tf.nn.relu(conv4_3x3)
            conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])
            #conv4_1x1_32 = batch_norm(conv4_1x1_32, conv4_1x1_32[0].shape[2], training)
            conv4_1x1_32 = tf.layers.batch_normalization(conv4_1x1_32, training=training, name='bn_conv4')

            #64
            conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
            conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
            network['conv4_1x1_32'] = conv4_1x1_32
    
        with tf.name_scope('conv6') as scope:
            conv5_1x1 = tf.nn.conv2d(conv4_1x1_32, _w['conv5_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_1x1 = tf.nn.bias_add(conv5_1x1, _b['conv5_1x1'])
            conv5_1x1 = tf.nn.relu(conv5_1x1)
            conv5_3x3 = tf.nn.conv2d(conv5_1x1, _w['conv5_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_3x3 = tf.nn.bias_add(conv5_3x3, _b['conv5_3x3'])
            conv5_3x3 = tf.nn.relu(conv5_3x3)
            conv5_1x1_64 = tf.nn.conv2d(conv5_3x3, _w['conv5_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_1x1_64 = tf.nn.bias_add(conv5_1x1_64, _b['conv5_1x1_64'])
            #conv5_1x1_64 = batch_norm(conv5_1x1_64, conv5_1x1_64[0].shape[2], training)
            conv5_1x1_64 = tf.layers.batch_normalization(conv5_1x1_64, training=training, name='bn_conv5')
            conv5_1x1_64 = tf.nn.relu(conv5_1x1_64)
            network['conv5_1x1_64'] = conv5_1x1_64
        
        with tf.name_scope('conv7') as scope:
            conv6_1x1 = tf.nn.conv2d(conv5_1x1_64, _w['conv6_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_1x1 = tf.nn.bias_add(conv6_1x1, _b['conv6_1x1'])
            conv6_1x1 = tf.nn.relu(conv6_1x1)
            conv6_3x3 = tf.nn.conv2d(conv6_1x1, _w['conv6_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_3x3 = tf.nn.bias_add(conv6_3x3, _b['conv6_3x3'])
            conv6_3x3 = tf.nn.relu(conv6_3x3)
            conv6_1x1_64 = tf.nn.conv2d(conv6_3x3, _w['conv6_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_1x1_64 = tf.nn.bias_add(conv6_1x1_64, _b['conv6_1x1_64'])
            #conv6_1x1_64 = batch_norm(conv6_1x1_64, conv6_1x1_64[0].shape[2], training)
            conv6_1x1_64 = tf.layers.batch_normalization(conv6_1x1_64, training=training, name='bn_conv6')
            
            #128
            conv6_1x1_64 = tf.concat([conv4_1x1_32, conv6_1x1_64],3)
            conv6_1x1_64 = tf.nn.relu(conv6_1x1_64)
            network['conv6_1x1_64'] = conv6_1x1_64
        
        with tf.name_scope('pool') as scope:
            pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            network['pool'] = pool
        
        with tf.name_scope('dense1') as scope:
            dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
            dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
            #dense1 = batch_norm(dense1, 1, training)
            dense1 =  tf.layers.batch_normalization(dense1, training=training, name='bn_dense')
            dense1 = tf.nn.relu(dense1)
            network['dense1'] = dense1
            
        with tf.name_scope('dropout') as scope:
            if training:
                dropout = tf.nn.dropout(dense1, 0.5, name='dropout')
            
        with tf.name_scope('logit') as scope:
            logit = tf.add(tf.matmul(dropout, _w['dense2']), _b['dense2'])
            network['logit'] = logit
        
    return network

# LOAD DATASET & CHECK LAYERS

In [8]:
images, labels = distorted_inputs(DATA_PATH, 64)

#labels = tf.cast(labels, tf.float32)
labels = tf.one_hot(indices=labels, depth=10, on_value=1, off_value=0, axis=1)

print(labels.shape)

out = ResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, images, weights, biases, 'ResNet', True)

for key, value in out.items():
    print (key, value)

Tensor("ReaderReadV2:1", shape=(), dtype=string)
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
(64, 10)
conv Tensor("ResNet/conv/Relu:0", shape=(64, 32, 32, 16), dtype=float32)
conv1_1x1_16 Tensor("ResNet/conv2/Relu_2:0", shape=(64, 32, 32, 16), dtype=float32)
conv2_1x1_16 Tensor("ResNet/conv3/Relu_2:0", shape=(64, 32, 32, 32), dtype=float32)
conv3_1x1_32 Tensor("ResNet/conv4/Relu_2:0", shape=(64, 32, 32, 32), dtype=float32)
conv4_1x1_32 Tensor("ResNet/conv5/Relu_2:0", shape=(64, 32, 32, 64), dtype=float32)
conv5_1x1_64 Tensor("ResNet/conv6/Relu_2:0", shape=(64, 32, 32, 64), dtype=float32)
conv6_1x1_64 Tensor("ResNet/conv7/Relu_2:0", shape=(64, 32, 32, 128), dtype=float32)
pool Tensor("ResNet/pool/AvgPool:0", shape=(64, 16, 16, 128), dtype=float32)
dense1 Tensor("ResNet/dense1/Relu:0", shape=(64, 1000), dtype=float32)
logit Tensor("ResNet/logit/Add:0", shape=(64, 10), dtype=float32)


# LOSS & OPTIMIZER

In [9]:
#loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=out['logit']))
tf.summary.scalar("loss", loss) #save loss 
print("LOSS FUNCTION")

#learning rate
#global_step = tf.Variable(0, trainable=False, name="global_step")
global_step = 0 #global_step: A scalar int32 or int64 Tensor or a Python number.  
#글로벌 스텝을 텐서로 쓰려면 tf.contrib를 사용해야 할 것 같다.. 
#train_op = tf.contrib.training.create_train_op(total_loss, optimizer, global_step=global_step) 이런느낌? 
learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                           global_step=global_step,
                                           decay_steps=300,
                                           decay_rate=LR_DECAY_RATE,
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate) #learning rate
print("LERANING RATE : {}".format(learning_rate))

#optimizer
adam = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
sgd = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(labels,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))
tf.summary.scalar("accuracy", accr) #save accuracy

LOSS FUNCTION
LERANING RATE : Tensor("Maximum_1:0", shape=(), dtype=float32)


<tf.Tensor 'accuracy:0' shape=() dtype=string>

# SESSION & SAVER & TENSORBOARD 

In [10]:
#SESSION INITIALIZE
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#SAVER
saver = tf.train.Saver(max_to_keep=3) #최근 3개까지만 저장
save_step = 100 #save for 100 epoch
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    
#restore checkpoint
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)
if checkpoint is not None:
    print(checkpoint)
    #saver.restore(sess, checkpoint)
    
#TENSOR BOARD
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(BOARD_PATH, sess.graph)

#QUEUE
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

print("initialized")

./checkpoint/res_0604/simple_res_net_0529.ckpt-310
initialized


# TRAINING

In [11]:
#training
train_loss_for_plot = []
train_acc_for_plot = []
test_loss_for_plot = []
test_acc_for_plot = []

print('start')
for epoch in range(EPOCHS):
    print(epoch)
    global_stop = epoch #lr decay
    nNumBatch = 0 # batch 수
    AvgBatchCost = 0 #cost 합산
    
    for i in range(int(NUM_BATCHES_PER_EPOCH)):
        nNumBatch += 1
        if epoch < EPOCHS * 0.5:
            _, tmp_cost = sess.run([adam, loss])
        else:
            #print('use sgd')
            _, tmp_cost = sess.run([sgd, loss])
            
        AvgBatchCost += tmp_cost
    
        if nNumBatch % 100 == 0:#print
            train_acc = sess.run(accr)
            print('\t[%d nNumBatch] train cost = %g, acc = %g' %(nNumBatch, AvgBatchCost/nNumBatch, train_acc ))
            lr = sess.run(learning_rate)
            print('\t\t learning rate = %g'%(lr))
            train_loss_for_plot.append(AvgBatchCost/nNumBatch)
            train_acc_for_plot.append(train_acc)
            
    if epoch % 10 == 0: #test
        save_path = saver.save(sess, SAVE_PATH + CHECKPOINT, global_step=epoch)
        print(save_path)
       
    

start
0
	[100 nNumBatch] train cost = 3.88073, acc = 0.234375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 2.99625, acc = 0.25
		 learning rate = 0.1
	[300 nNumBatch] train cost = 2.69297, acc = 0.171875
		 learning rate = 0.1
	[400 nNumBatch] train cost = 2.539, acc = 0.1875
		 learning rate = 0.1
	[500 nNumBatch] train cost = 2.45245, acc = 0.1875
		 learning rate = 0.1
	[600 nNumBatch] train cost = 2.39192, acc = 0.203125
		 learning rate = 0.1
	[700 nNumBatch] train cost = 2.34652, acc = 0.140625
		 learning rate = 0.1
./checkpoint/res_0604/simple_res_net_0529.ckpt-0
1
	[100 nNumBatch] train cost = 2.08552, acc = 0.234375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 2.06523, acc = 0.171875
		 learning rate = 0.1
	[300 nNumBatch] train cost = 2.05352, acc = 0.296875
		 learning rate = 0.1
	[400 nNumBatch] train cost = 2.04272, acc = 0.265625
		 learning rate = 0.1
	[500 nNumBatch] train cost = 2.03182, acc = 0.1875
		 learning rate = 0.1
	[600 nNumBatch] train cost =

	[300 nNumBatch] train cost = 1.68406, acc = 0.203125
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.68798, acc = 0.359375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.6846, acc = 0.265625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.68335, acc = 0.328125
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.68413, acc = 0.359375
		 learning rate = 0.1
16
	[100 nNumBatch] train cost = 1.69485, acc = 0.28125
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.69354, acc = 0.34375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.68636, acc = 0.453125
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.69173, acc = 0.328125
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.69104, acc = 0.328125
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.68997, acc = 0.21875
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.68905, acc = 0.328125
		 learning rate = 0.1
17
	[100 nNumBatch] train cost = 1.68199, acc = 0.359375
		 learning rate = 0

	[500 nNumBatch] train cost = 1.63767, acc = 0.4375
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.63628, acc = 0.5
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.63474, acc = 0.296875
		 learning rate = 0.1
./checkpoint/res_0604/simple_res_net_0529.ckpt-30
31
	[100 nNumBatch] train cost = 1.65301, acc = 0.390625
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.63594, acc = 0.421875
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.63978, acc = 0.375
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.64042, acc = 0.265625
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.64086, acc = 0.375
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.6413, acc = 0.390625
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.64184, acc = 0.453125
		 learning rate = 0.1
32
	[100 nNumBatch] train cost = 1.61362, acc = 0.390625
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.6164, acc = 0.453125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.617

	[600 nNumBatch] train cost = 1.6797, acc = 0.390625
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.6758, acc = 0.40625
		 learning rate = 0.1
46
	[100 nNumBatch] train cost = 1.64805, acc = 0.375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.66044, acc = 0.375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.65618, acc = 0.359375
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.65469, acc = 0.28125
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.65476, acc = 0.421875
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.65323, acc = 0.21875
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.65238, acc = 0.359375
		 learning rate = 0.1
47
	[100 nNumBatch] train cost = 1.65766, acc = 0.25
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.6643, acc = 0.359375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.66994, acc = 0.296875
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.67428, acc = 0.296875
		 learning rate = 0.1
	[500 nNu

./checkpoint/res_0604/simple_res_net_0529.ckpt-60
61
	[100 nNumBatch] train cost = 1.63906, acc = 0.328125
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.64673, acc = 0.4375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.64432, acc = 0.390625
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.64002, acc = 0.375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.63784, acc = 0.421875
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.63613, acc = 0.40625
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.63694, acc = 0.421875
		 learning rate = 0.1
62
	[100 nNumBatch] train cost = 1.6272, acc = 0.40625
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.6239, acc = 0.34375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.6308, acc = 0.296875
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.63081, acc = 0.359375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.63296, acc = 0.46875
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.

...

928
	[100 nNumBatch] train cost = 1.63452, acc = 0.421875
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.61985, acc = 0.359375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.62117, acc = 0.359375
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.62669, acc = 0.3125
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.62643, acc = 0.359375
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.62538, acc = 0.453125
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62584, acc = 0.375
		 learning rate = 0.1
929
	[100 nNumBatch] train cost = 1.62812, acc = 0.4375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.61439, acc = 0.375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.61721, acc = 0.453125
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.61736, acc = 0.359375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.61761, acc = 0.328125
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.62096, acc = 0.390625
		 learning rate = 0.1
	

	[200 nNumBatch] train cost = 1.61505, acc = 0.28125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.62271, acc = 0.375
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.6291, acc = 0.421875
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.62833, acc = 0.40625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.6271, acc = 0.4375
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62861, acc = 0.5
		 learning rate = 0.1
944
	[100 nNumBatch] train cost = 1.63789, acc = 0.4375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.64311, acc = 0.453125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.63655, acc = 0.25
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.63528, acc = 0.34375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.63697, acc = 0.328125
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.63497, acc = 0.46875
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.63248, acc = 0.34375
		 learning rate = 0.1
945
	[100 nNumBatc

	[400 nNumBatch] train cost = 1.62887, acc = 0.484375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.63048, acc = 0.390625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.62523, acc = 0.296875
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62857, acc = 0.34375
		 learning rate = 0.1
959
	[100 nNumBatch] train cost = 1.60154, acc = 0.453125
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.61127, acc = 0.375
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.62195, acc = 0.46875
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.6263, acc = 0.34375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.62725, acc = 0.359375
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.62532, acc = 0.28125
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62432, acc = 0.453125
		 learning rate = 0.1
960
	[100 nNumBatch] train cost = 1.62285, acc = 0.328125
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.62702, acc = 0.359375
		 learning rate = 0.1

	[500 nNumBatch] train cost = 1.62767, acc = 0.40625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.6285, acc = 0.40625
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62926, acc = 0.390625
		 learning rate = 0.1
974
	[100 nNumBatch] train cost = 1.62251, acc = 0.34375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.62338, acc = 0.421875
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.62276, acc = 0.453125
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.62214, acc = 0.375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.6285, acc = 0.265625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.62957, acc = 0.28125
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.63158, acc = 0.375
		 learning rate = 0.1
975
	[100 nNumBatch] train cost = 1.64683, acc = 0.234375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.63715, acc = 0.3125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.63365, acc = 0.46875
		 learning rate = 0.1
	[400 

	[700 nNumBatch] train cost = 1.6269, acc = 0.359375
		 learning rate = 0.1
989
	[100 nNumBatch] train cost = 1.64873, acc = 0.234375
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.64116, acc = 0.3125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.63227, acc = 0.375
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.62737, acc = 0.359375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.6235, acc = 0.40625
		 learning rate = 0.1
	[600 nNumBatch] train cost = 1.6228, acc = 0.296875
		 learning rate = 0.1
	[700 nNumBatch] train cost = 1.62196, acc = 0.34375
		 learning rate = 0.1
990
	[100 nNumBatch] train cost = 1.63995, acc = 0.296875
		 learning rate = 0.1
	[200 nNumBatch] train cost = 1.62977, acc = 0.453125
		 learning rate = 0.1
	[300 nNumBatch] train cost = 1.6248, acc = 0.328125
		 learning rate = 0.1
	[400 nNumBatch] train cost = 1.62562, acc = 0.4375
		 learning rate = 0.1
	[500 nNumBatch] train cost = 1.62243, acc = 0.390625
		 learning rate = 0.1
	[60

In [12]:
coord.request_stop()
coord.join(thresds)
print('thread is stopped')
sess.close()
print('session closed')

NameError: name 'thresds' is not defined

In [ ]:
# test 부분을 어디에 어떻게 넣어야 하나..

# PLOT

In [ ]:
print(train_loss_for_plot, file=train_loss_out)
print(train_acc_for_plot, file=train_accr_out)

print(test_loss_for_plot, file=test_loss_out)
print(test_acc_for_plot, file=test_accr_out)

In [ ]:
#LOSS
x = np.linspace(0, len(y), len(y))
y = loss_for_plot
plt.plot(x,y, color="red")
plt.plot()
plt.xlabel("batch")
plt.ylabel("loss")
plt.title("LOSS")

In [ ]:
#ACCURACY
x = np.linspace(0, len(y_ac), len(y_ac))
y_ac = acc_for_plot
plt.plot(x,y_ac, color="blue")
plt.plot()
plt.xlabel("batch")
plt.ylabel("accuracy")
plt.title("ACCURACY")

# SHOW FEATURES

In [ ]:
example_image = 불러올 이미지

In [ ]:
out = ResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, images, weights, biases, 'ResNet', True) 이런식으로 불러올것

In [ ]:
for layer_name in features:
    if layer_name.startswith('dense'): break
    print(layer_name)
    plt.matshow(example_net[layer_name].eval(sess=sess)[0, :, :, 0], cmap=plt.cm.gray)
    plt.show();
보여줄것

1. test 어떻게 넣을지
2. 드랍아웃을 적용하긴 했는데 맞는건지
3. 트레이닝 끝난 후 피쳐 이미지 보여주는것
4. 러닝레이트 디케이 되는걸 확인하기
5. 미쓰클레시피케이션 된놈. 레이블 찾아서 보여주기
6. 텐서보드 이쁘게 

In [ ]:
images_test, labels_test = inputs(True, DATA_PATH, 64) #Eval

In [ ]:
labels_test = tf.one_hot(indices=labels_test, depth=10, on_value=1, off_value=0, axis=1)

In [ ]:
out_test = ResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, images_test, weights, biases, 'ResNet_Test') 

In [ ]:
corr_test = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(labels_test,1))
accr_test = tf.reduce_mean(tf.cast(corr_test, "float"))

In [ ]:
loss_test = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels_test, logits=out['logit']))

In [ ]:
# 드랍아웃 추가 
# 트레이닝 끝나고 테스트 ㄱ 
